In [19]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import swan_vis as swan 
from scipy import sparse
import sqlite3

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

## Create a GTF with all polya, complete, detected transcripts AND all known transcripts

In [20]:
df = pd.read_csv('human_talon_abundance_filtered.tsv', sep='\t')
_, inds = get_tpm_table(df, 
                        how='iso', 
                        sample='all', 
                        min_tpm=1,
                        gene_subset='polya',
                        nov=['Known', 'NIC', 'NNC'])
df = df.loc[df.annot_transcript_id.isin(inds)]
print(len(df.loc[df.transcript_novelty != 'Known'].index))
df = df[['gene_ID', 'transcript_ID']]

Calculating iso TPM values
Subsetting for novelty categories ['Known', 'NIC', 'NNC']
Subsetting for polya genes


/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/scripts/utils.py:1099: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tpm_col] = (df[d]*1000000)/df[total_col]
/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/scripts/utils.py:1098: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[total_col] = df[d].sum()


Enforcing minimum TPM
Total # isos detected: 194503
# isos >= 1 tpm: 175394
Applying gene type and novelty subset
Number of isos reported: 111071
35682


In [21]:
df.to_csv('human_known_nic_nnc_detected_talon_pass_list.csv', header=None, index=False)

In [11]:
db = 'human.db'
with sqlite3.connect(db) as conn:
    query = """SELECT DISTINCT t.gene_ID, t.transcript_ID
                    FROM transcripts as t
                    LEFT JOIN transcript_annotations as ta
                        ON ta.ID = t.transcript_ID
                    WHERE (ta.attribute = 'transcript_status'
                        AND ta.value = 'KNOWN')
            """
    annot = pd.read_sql_query(query, conn)

df = pd.concat([df, annot])
df.drop_duplicates(inplace=True)

In [12]:
df.to_csv('human_complete_pass_list.csv', header=None, index=False)

Traceback (most recent call last):
  File "/Users/fairliereese/miniconda3/bin/talon_create_GTF", line 33, in <module>
    sys.exit(load_entry_point('talon', 'console_scripts', 'talon_create_GTF')())
  File "/Users/fairliereese/Documents/programming/mortazavi_lab/bin/TALON/src/talon/post/create_GTF_from_database.py", line 617, in main
    check_annot_validity(annot, database)
  File "/Users/fairliereese/Documents/programming/mortazavi_lab/bin/TALON/src/talon/post/create_GTF_from_database.py", line 562, in check_annot_validity
    cursor.execute("SELECT DISTINCT annot_name FROM gene_annotations")
sqlite3.OperationalError: no such table: gene_annotations
